# Train with SageMaker ObjectToVec algorithm


1. [Background](#Background)
1. [Data exploration and preparation](#Data-exploration-and-preparation)
1. [Rating prediction task](#Rating-prediction-task)
1. [Recommendation task](#Recommendation-task)
1. [Movie retrieval in the embedding space](#Movie-retrieval-in-the-embedding-space)

# Background

### ObjectToVec
*Object2Vec* is a highly customizable multi-purpose algorithm that can learn embeddings of pairs of objects. The embeddings are learned such that it preserves their pairwise **similarities** in the original space.
- **Similarity** is user-defined: users need to provide the algorithm with pairs of objects that they define as similar (1) or dissimilar (0); alternatively, the users can define similarity in a continuous sense (provide a real-valued similarity score)
- The learned embeddings can be used to efficiently compute nearest neighbors of objects, as well as to visualize natural clusters of related objects in the embedding space. In addition, the embeddings can also be used as features of the corresponding objects in downstream supervised tasks such as classification or regression

## In this notebook example:
We demonstrate how Object2Vec can be used to solve problems arising in recommendation systems. Specifically,

- We provide the algorithm with (UserID, MovieID) pairs; for each such pair, we also provide a "label" that tells the algorithm whether the user and movie are similar or not

     * When the labels are real-valued, we use the algorithm to predict the exact ratings of a movie given a user
     * When the labels are binary, we use the algorithm to recommendation movies to users

- The diagram below shows the customization of our model to the problem of predicting movie ratings, using a dataset that provides `(UserID, ItemID, Rating)` samples. Here, ratings are real-valued

<img style="float:middle" src="https://github.com/aws/amazon-sagemaker-examples/blob/ac7f655c600f1fed7922339b65982cb7964a3b82/introduction_to_amazon_algorithms/object2vec_movie_recommendation/image_ml_rating.png?raw=true" width="480">

### Use cases

- Task 1: Movie recommendation (classification)
- Task 2: Nearest-neighbor movie retrieval in the learned embedding space

### Before running the notebook
- Please use a Python 3 kernel for the notebook
- Please make sure you have `jsonlines` package installed (if not, you can run the command below to install it)

In [2]:
!pip install jsonlines


  Using cached jsonlines-1.2.0-py2.py3-none-any.whl (7.6 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [6]:
# Import packages
import os
import pandas as pd
import numpy as np
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE

import json
%matplotlib inline
import matplotlib.pyplot as plt

import helper
import importlib
_ = importlib.reload(helper)


import copy
import random


In [7]:
algo_name = 'object2vec'
dataset_name = 'ml-latest-small' #, 'book-crossing', 'ml-100k'


In [8]:
from sagemaker import get_execution_role
from sagemaker.session import Session

sagemaker_session = Session()
region = sagemaker_session.boto_session.region_name

# S3 bucket for saving files. Feel free to redefine this variable to the bucket of your choice.
bucket = sagemaker_session.default_bucket()

prefix = f'sagemaker/DEMO-recsys-{algo_name}-{dataset_name}'

input_prefix = f'{prefix}/train'
output_prefix = f'{prefix}/output'

# Bucket location where your custom code will be saved in the tar.gz format.
custom_code_upload_location = f's3://{bucket}/{prefix}/code'

# Bucket location where results of model training are saved.
model_artifacts_location = f's3://{bucket}/{prefix}/artifacts'

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()


In [9]:
df_train = helper.get_csv(dataset_name, "interactions_train.csv.gz")
df_test = helper.get_csv(dataset_name, "interactions_test.csv.gz")
df_user_index = helper.get_csv(dataset_name, "user_index.csv.gz")
df_item_index = helper.get_csv(dataset_name, "item_index.csv.gz")


In [12]:
import object2vec


## Load data and shuffle
train_data_list = object2vec.load_data_from_df(df_train)
random.shuffle(train_data_list)
validation_data_list = object2vec.load_data_from_df(df_test)
random.shuffle(validation_data_list)

to_users_dict, to_movies_dict = object2vec.df_to_augmented_data_dict(df_train)

There are 94184 ratings
The ratings have mean: 3.52, median: 3.5, and variance: 1.07
There are 610 unique users and 4972 unique movies
There are 610 ratings
The ratings have mean: 3.53, median: 4.0, and variance: 1.35
There are 610 unique users and 472 unique movies


In [14]:
## Save training and validation data locally for rating-prediction (regression) task

object2vec.write_data_list_to_jsonl(copy.deepcopy(train_data_list), f'data/{dataset_name}/train_r.jsonl')
object2vec.write_data_list_to_jsonl(copy.deepcopy(validation_data_list), f'data/{dataset_name}/validation_r.jsonl')

Created data/ml-latest-small/train_r.jsonl jsonline file
Created data/ml-latest-small/validation_r.jsonl jsonline file


In [16]:
## Save training and validation data locally for recommendation (classification) task

### binarize the data 

train_c = object2vec.get_binarized_label(copy.deepcopy(train_data_list), 3.0)
valid_c = object2vec.get_binarized_label(copy.deepcopy(validation_data_list), 3.0)

object2vec.write_data_list_to_jsonl(train_c, f'data/{dataset_name}/train_c.jsonl')
object2vec.write_data_list_to_jsonl(valid_c, f'data/{dataset_name}/validation_c.jsonl')

Created data/ml-latest-small/train_c.jsonl jsonline file
Created data/ml-latest-small/validation_c.jsonl jsonline file


**We check whether the two classes are balanced after binarization**

In [17]:
train_c_label = [row['label'] for row in train_c]
valid_c_label = [row['label'] for row in valid_c]

print("There are {} fraction of positive ratings in train_c.jsonl".format(
                                np.count_nonzero(train_c_label)/len(train_c_label)))
print("There are {} fraction of positive ratings in validation_c.jsonl".format(
                                np.sum(valid_c_label)/len(valid_c_label)))

There are 0.6173660069650896 fraction of positive ratings in train_c.jsonl
There are 0.6081967213114754 fraction of positive ratings in validation_c.jsonl


# Rating prediction task 

In [18]:
def get_mse_loss(res, labels):
    if type(res) is dict:
        res = res['predictions']
    assert len(res)==len(labels), 'result and label length mismatch!'
    loss = 0
    for row, label in zip(res, labels):
        if type(row)is dict:
            loss += (row['scores'][0]-label)**2
        else:
            loss += (row-label)**2
    return round(loss/float(len(labels)), 2)

In [20]:
valid_r_data, valid_r_label = object2vec.data_list_to_inference_format(
    copy.deepcopy(
        validation_data_list
    ),
    binarize=False,
)

#### We first test the problem on two baseline algorithms

## Baseline 1

A naive approach to predict movie ratings on unseen data is to use the global average of the user predictions in the training data

In [21]:
train_r_label = [row['label'] for row in copy.deepcopy(train_data_list)]

bs1_prediction = round(np.mean(train_r_label), 2)
print('The Baseline 1 (global rating average) prediction is {}'.format(bs1_prediction))
print(
    "The validation mse loss of the Baseline 1 is {}".format(
        get_mse_loss(
            len(valid_r_label)*[bs1_prediction],
            valid_r_label,
        )
    )
)

The Baseline 1 (global rating average) prediction is 3.52
The validation mse loss of the Baseline 1 is 1.35


## Baseline 2

Now we use a better baseline, which is to perform prediction on unseen data based on the user-averaged ratings of movies on training data

In [22]:
def bs2_predictor(test_data, user_dict, is_classification=False, thres=3):
    test_data = copy.deepcopy(test_data['instances'])
    predictions = list()
    for row in test_data:
        userID = int(row["in0"][0])
        # predict movie ID based on local average of user's prediction
        local_movies, local_ratings = zip(*user_dict[userID])
        local_ratings = [float(score) for score in local_ratings]
        predictions.append(np.mean(local_ratings))
        if is_classification:
            predictions[-1] = int(predictions[-1] > 3)
    return predictions

In [23]:
bs2_prediction = bs2_predictor(valid_r_data, to_users_dict, is_classification=False)
print(
    "The validation loss of the Baseline 2 (user-based rating average) is {}".format(
        get_mse_loss(
            bs2_prediction,
            valid_r_label,
        )
    )
)

The validation loss of the Baseline 2 (user-based rating average) is 1.22


Next, we will use *Object2Vec* to predict the movie ratings

## Model training and inference

#### Define S3 bucket that hosts data and model, and upload data to S3

#### Upload data to S3 and make data paths

In [25]:
import boto3
from sagemaker.session import s3_input

s3_client = boto3.client('s3')
input_paths = {}
output_path = os.path.join('s3://', bucket, output_prefix)

for data_name in ['train', 'validation']:
    fname = '{}_r.jsonl'.format(data_name)
    object_key = f'{input_prefix}/{fname}'
    data_path = f's3://{bucket}/{object_key}'
    s3_client.upload_file(f'data/{dataset_name}/{fname}', bucket, object_key)
    input_paths[data_name] = s3_input(data_path, distribution='ShardedByS3Key', content_type='application/jsonlines')
    print('Uploaded {} data to {} and defined input path'.format(data_name, data_path))

print('Trained model will be saved at', output_path)

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Uploaded train data to s3://sagemaker-us-east-1-533025023261/sagemaker/DEMO-recsys-object2vec-ml-latest-small/train/train_r.jsonl and defined input path
Uploaded validation data to s3://sagemaker-us-east-1-533025023261/sagemaker/DEMO-recsys-object2vec-ml-latest-small/train/validation_r.jsonl and defined input path
Trained model will be saved at s3://sagemaker-us-east-1-533025023261/sagemaker/DEMO-recsys-object2vec-ml-latest-small/output


### Get ObjectToVec algorithm image

In [ ]:
## Get docker image of ObjectToVec algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'object2vec')


### Training

#### We first define training hyperparameters

In [266]:
hyperparameters = {
    "_kvstore": "device",
    "_num_gpus": "auto",
    "_num_kv_servers": "auto",
    "bucket_width": 0,
    "early_stopping_patience": 3,
    "early_stopping_tolerance": 0.01,
    "enc0_cnn_filter_width": 3,
    "enc0_layers": "auto",
    "enc0_max_seq_len": 1,
    "enc0_network": "pooled_embedding",
    "enc0_token_embedding_dim": 300,
    "enc0_vocab_size": 944,
    "enc1_layers": "auto",
    "enc1_max_seq_len": 1,
    "enc1_network": "pooled_embedding",
    "enc1_token_embedding_dim": 300,
    "enc1_vocab_size": 1684,
    "enc_dim": 1024,
    "epochs": 20,
    "learning_rate": 0.001,
    "mini_batch_size": 64,
    "mlp_activation": "tanh",
    "mlp_dim": 256,
    "mlp_layers": 1,
    "num_classes": 2,
    "optimizer": "adam",
    "output_layer": "mean_squared_error"
}

train_instance_type = "ml.p3.2xlarge"
use_spot_instances = True
max_run = 600
max_wait = 1200 if use_spot_instances else None


In [267]:
from sagemaker.estimator import Estimator

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    instance_type=train_instance_type,
    instance_count=1,
    output_path=output_path,
    sagemaker_session=sess
)

## set hyperparameters
estimator.set_hyperparameters(**hyperparameters)

## train the model
estimator.fit(
    input_paths,
    wait=False,
)
training_job_name = estimator.latest_training_job.name

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: object2vec-2020-12-13-19-37-26-122


In [268]:
attached_estimator = Estimator.attach(training_job_name)


2020-12-13 19:37:26 Starting - Starting the training job
2020-12-13 19:37:27 Starting - Launching requested ML instances.............
2020-12-13 19:38:39 Starting - Preparing the instances for training..........
2020-12-13 19:39:33 Downloading - Downloading input data
2020-12-13 19:39:40 Training - Downloading the training image........
2020-12-13 19:40:24 Training - Training image download completed. Training in progress........
2020-12-13 19:41:05 Uploading - Uploading generated training model.
2020-12-13 19:41:14 Completed - Training job completed


We have seen that we can upload train (validation) data through the input data channel, and the algorithm will print out train (validation) evaluation metric during training. In addition, the algorithm uses the validation metric to perform early stopping. 

What if we want to send additional unlabeled data to the algorithm and get predictions from the trained model?
This step is called *inference* in the Sagemaker framework. Next, we demonstrate how to use a trained model to perform inference on unseen data points.

## Inference using trained model

Create and deploy the model

In [273]:
# create a model using the trained algorithm
regression_model = attached_estimator.create_model()

In [ ]:
#import numpy as np
from sagemaker.predictor import json_serializer, json_deserializer

# deploy the model
predictor = regression_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    serializer=json_serializer,
    deserializer=json_deserializer,
    content_type='application/json',
)


Below we send validation data (without labels) to the deployed endpoint for inference. We will see that the resulting prediction error we get from post-training inference matches the best validation error from the training log in the console above (up to floating point error). If you follow the training instruction and parameter setup, you should get mean squared error on the validation set approximately 0.91.

In [ ]:
# Send data to the endpoint to get predictions
prediction = predictor.predict(valid_r_data)

print("The mean squared error on validation set is %.3f" %get_mse_loss(prediction, valid_r_label))

### Comparison against popular libraries

Below we provide a chart that compares the performance of *Object2Vec* against several algorithms implemented by popular recommendation system libraries (LibRec https://www.librec.net/ and scikit-surprise http://surpriselib.com/). The error metric we use in the chart is **root mean squared** (RMSE) instead of MSE, so that our result can be compared against the reported results in the aforementioned libraries.

<img src="https://github.com/aws/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/object2vec_movie_recommendation/ml-experiment-plot.png?raw=true" width="400">

# Recommendation task 

In this section, we showcase how to use *Object2Vec* to recommend movies, using the binarized rating labels. Here, if a movie rating label for a given user is binarized to `1`, then it means that the movie should be recommended to the user; otherwise, the label is binarized to `0`. The binarized data set is already obtained in the preprocessing section, so we will proceed to apply the algorithm.

We upload the binarized datasets for classification task to S3

In [ ]:
for data_name in ['train', 'validation']:
    fname = '{}_c.jsonl'.format(data_name)
    pre_key = os.path.join(input_prefix, 'recommendation', f"{data_name}")
    data_path = os.path.join('s3://', bucket, pre_key, fname)
    s3_client.upload_file(fname, bucket, os.path.join(pre_key, fname))
    input_paths[data_name] = s3_input(data_path, distribution='ShardedByS3Key', content_type='application/jsonlines')
    print('Uploaded data to {}'.format(data_path))

Since we already get the algorithm image from the regression task, we can directly start training

In [ ]:
from sagemaker.session import s3_input

hyperparameters_c = {
    "_kvstore": "device",
    "_num_gpus": "auto",
    "_num_kv_servers": "auto",
    "bucket_width": 0,
    "early_stopping_patience": 3, 
    "early_stopping_tolerance": 0.01,
    "enc0_cnn_filter_width": 3,
    "enc0_layers": "auto",
    "enc0_max_seq_len": 1,
    "enc0_network": "pooled_embedding",
    "enc0_token_embedding_dim": 300,
    "enc0_vocab_size": 944,
    "enc1_cnn_filter_width": 3,
    "enc1_layers": "auto",
    "enc1_max_seq_len": 1,
    "enc1_network": "pooled_embedding",
    "enc1_token_embedding_dim": 300,
    "enc1_vocab_size": 1684,
    "enc_dim": 2048,
    "epochs": 20,
    "learning_rate": 0.001,
    "mini_batch_size": 2048,
    "mlp_activation": "relu",
    "mlp_dim": 1024,
    "mlp_layers": 1,
    "num_classes": 2,
    "optimizer": "adam",
    "output_layer": "softmax"
}

In [ ]:
## get estimator
classifier = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.p2.xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sess)

## set hyperparameters
classifier.set_hyperparameters(**hyperparameters_c)

## train, tune, and test the model
classifier.fit(input_paths)

Again, we can create, deploy, and validate the model after training

In [ ]:
classification_model = classifier.create_model(
                        serializer=json_serializer,
                        deserializer=json_deserializer,
                        content_type='application/json')

predictor_2 = classification_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
valid_c_data, valid_c_label = data_list_to_inference_format(copy.deepcopy(validation_data_list), 
                                                            label_thres=3, binarize=True)
predictions = predictor_2.predict(valid_c_data)

In [ ]:
def get_class_accuracy(res, labels, thres):
    if type(res) is dict:
        res = res['predictions']
    assert len(res)==len(labels), 'result and label length mismatch!'
    accuracy = 0
    for row, label in zip(res, labels):
        if type(row) is dict:
            if row['scores'][1] > thres:
                prediction = 1
            else: 
                prediction = 0
            if label > thres:
                label = 1
            else:
                label = 0
            accuracy += 1 - (prediction - label)**2
    return accuracy / float(len(res))

print("The accuracy on the binarized validation set is %.3f" %get_class_accuracy(predictions, valid_c_label, 0.5))

The accuracy on validation set you would get should be approximately 0.704.

## Movie retrieval in the embedding space

Since *Object2Vec* transforms user and movie ID's into embeddings as part of the training process. After training, it obtains user and movie embeddings in the left and right encoders, respectively. Intuitively, the embeddings should be tuned by the algorithm in a way that facilitates the supervised learning task: since for a specific user, similar movies should have similar ratings, we expect that similar movies should be **close-by** in the embedding space.

In this section, we demonstrate how to find the nearest-neighbor (in Euclidean distance) of a given movie ID, among all movie ID's.

In [ ]:
def get_movie_embedding_dict(movie_ids, trained_model):
    input_instances = list()
    for s_id in movie_ids:
        input_instances.append({'in1': [s_id]})
    data = {'instances': input_instances}
    movie_embeddings = trained_model.predict(data)
    embedding_dict = {}
    for s_id, row in zip(movie_ids, movie_embeddings['predictions']):
        embedding_dict[s_id] = np.array(row['embeddings'])
    return embedding_dict


def load_movie_id_name_map(item_file):
    movieID_name_map = {}
    with open(item_file, 'r', encoding="ISO-8859-1") as f:
        for row in f.readlines():
            row = row.strip()
            split = row.split('|')
            movie_id = split[0]
            movie_name = split[1]
            sparse_tags = split[-19:]
            movieID_name_map[int(movie_id)] = movie_name 
    return movieID_name_map

            
def get_nn_of_movie(movie_id, candidate_movie_ids, embedding_dict):
    movie_emb = embedding_dict[movie_id]
    min_dist = float('Inf')
    best_id = candidate_movie_ids[0]
    for idx, m_id in enumerate(candidate_movie_ids):
        candidate_emb = embedding_dict[m_id]
        curr_dist = np.linalg.norm(candidate_emb - movie_emb)
        if curr_dist < min_dist:
            best_id = m_id
            min_dist = curr_dist
    return best_id, min_dist


def get_unique_movie_ids(data_list):
    unique_movie_ids = set()
    for row in data_list:
        unique_movie_ids.add(row['in1'][0])
    return list(unique_movie_ids)

In [ ]:
train_data_list = load_csv_data(train_path, '\t', verbose=False)
unique_movie_ids = get_unique_movie_ids(train_data_list)
embedding_dict = get_movie_embedding_dict(unique_movie_ids, predictor_2)
candidate_movie_ids = unique_movie_ids.copy()

Using the script below, you can check out what is the closest movie to any movie in the data set. Last time we ran it, the closest movie to `Terminator, The (1984)` in the embedding space was `Die Hard (1988)`. Note that, the result will likely differ slightly across different runs of the algorithm, due to randomness in initialization of model parameters.

- Just plug in the movie id you want to examine 
   - For example, the movie ID for Terminator is 195; you can find the movie name and ID pair in the `u.item` file
- Note that, the result will likely differ across different runs of the algorithm, due to inherent randomness.

In [ ]:
movie_id_to_examine = '<movie id>' # Customize the movie ID you want to examine

In [ ]:
candidate_movie_ids.remove(movie_id_to_examine)
best_id, min_dist = get_nn_of_movie(movie_id_to_examine, candidate_movie_ids, embedding_dict)
movieID_name_map = load_movie_id_name_map('ml-100k/u.item')
print('The closest movie to {} in the embedding space is {}'.format(movieID_name_map[movie_id_to_examine],
                                                                  movieID_name_map[best_id]))
candidate_movie_ids.append(movie_id_to_examine)

It is recommended to always delete the endpoints used for hosting the model

In [ ]:
## clean up
sess.delete_endpoint(predictor.endpoint)
sess.delete_endpoint(predictor_2.endpoint)